<h1><center>Ind vs Pak Cricket Hackathon

In [1]:
!pip3 install pycaret --user

In [28]:
#Importing the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from pycaret.regression import *
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV

In [3]:
#Reading the training file
df = pd.read_csv('train-file.csv')
df.head()

,player_id,player_name,runs_scored,wickets,runs_conceded,catches,stumpings,match_date,opposition,match_id
0,1,Rohit Sharma,DNB,-,-,1,0,23 Jun 2007,v Ireland Belfast,1
1,1,Rohit Sharma,8,0,3,1,0,26 Jun 2007,v South Africa Belfast,2
2,1,Rohit Sharma,1,-,-,0,0,5 Oct 2007,v Australia Hyderabad,3
3,1,Rohit Sharma,52,-,-,1,0,18 Nov 2007,v Pakistan Jaipur,4
4,1,Rohit Sharma,29,-,-,0,0,3 Feb 2008,v Australia Brisbane,5


In [4]:
#There are features in the dataframe like runs_conceded, catches and stumping which cannot be obtained for a future match and therefore removing it from the dataframe
df.drop(columns=['runs_conceded','catches','stumpings'],inplace=True)
df.shape

(2021, 7)

In [5]:
#Creating new feature opposition team to generate and save team name from the feature opposition
international_cricket_teams = ["Australia", "Bangladesh", "England", "India", "New Zealand", "Pakistan",
                               "South Africa", "Sri Lanka", "West Indies", "Zimbabwe","Ireland",
                              "Hong Kong", "Afghanistan", "Nepal","Netherlands","U.A.E."]
df['opposition_team'] = df['opposition'].apply(lambda x : [i for i in international_cricket_teams if i in x][0])
df['opposition_team'].unique()

array(['Ireland', 'South Africa', 'Australia', 'Pakistan', 'Sri Lanka',
       'Bangladesh', 'Hong Kong', 'England', 'New Zealand', 'West Indies',
       'Zimbabwe', 'Afghanistan', 'U.A.E.', 'India', 'Nepal',
       'Netherlands'], dtype=object)

In [6]:
#Creating new feature location to generate and save the match location from the feature opposition
df['location'] = df.apply(lambda x : x['opposition'].replace('v ' + str(x['opposition_team']) + ' ',''), axis = 1)
df['location'].unique()

array(['Belfast', 'Hyderabad', 'Jaipur', 'Brisbane', 'Melbourne',
       'Canberra', 'Adelaide', 'Sydney', 'Hobart', 'Mirpur', 'Karachi',
       'Dambulla', 'Colombo (RPS)', 'Rajkot', 'Indore', 'Kanpur',
       'Cuttack', 'Hamilton', 'Auckland', 'Kingston', 'Gros Islet',
       'Ahmedabad', 'Bulawayo', 'Harare', 'Bengaluru', 'Chennai',
       'Durban', 'Johannesburg', 'Cape Town', 'Gqeberha', 'Centurion',
       'Port of Spain', 'North Sound', 'Chester-le-Street',
       'Visakhapatnam', 'Perth', 'Hambantota', 'Pallekele', 'Mohali',
       'Dharamsala', 'Cardiff', 'The Oval', 'Birmingham', 'Pune',
       'Ranchi', 'Nagpur', 'Kochi', 'Napier', 'Wellington', 'Fatullah',
       'Eden Gardens', 'Wankhede', 'Delhi', 'Nottingham', "Lord's",
       'Leeds', 'Dubai (DSC)', 'Guwahati', 'Brabourne',
       'Thiruvananthapuram', 'Mount Maunganui', 'Southampton',
       'Manchester', 'Providence', 'Raipur', 'Bridgetown', 'Tarouba',
       'Lucknow', 'Christchurch', 'Vadodara', 'Gwalior', 'Paarl',


In [7]:
df.drop(columns = 'opposition',inplace=True)

In [8]:
df.runs_scored.unique()

array(['DNB', '8', '1', '52', '29', '0', '39*', '70*', '24', '3*', '66',
       '2', '9', '26', '11', '0*', '22', '58', '22*', '3', '19', '12',
       '37', '32', '25', '18', '54', '31', '11*', '28', '8*', '25*', '4*',
       '15', '60*', '43*', '4', '7', 'TDNB', '48', '57', '36', '114',
       '61*', '101*', '82', '13', '51', '68', '38', '6', '69', '10', '41',
       '20', '5', '44', '23', '87*', '68*', '7*', '81', '86*', '39', '94',
       '55', '72', '90*', '117', '9*', '95', '27', '23*', '21', '80',
       '77', '24*', '30*', '33', '1*', '14', '108', '183', '106', '128*',
       '2*', '31*', '83', '21*', '65', '47*', '58*', '43', '33*', '60',
       '5*', '49*', '46', '102', '48*', '115', '6*', '64*', '42', '61',
       '141*', '100*', '79', '115*', '209', '86', '99', '123', '78',
       '66*', '62*', '14*', '136', '56', '52*', '18*', '10*', '264',
       '139*', '138', '107', '57*', '16*', '64', '44*', '16', '137', '34',
       '63', '150', '171*', '91', '124', '59', '85*', '154*'

In [9]:
for i in ['runs_scored', 'wickets']:
    for j in range(len(df[i])):
        df[i][j] = str(df[i][j]).replace('TDNB','0')
        df[i][j] = str(df[i][j]).replace('DNB','0')
        df[i][j] = str(df[i][j]).replace('absent','0')
        df[i][j] = str(df[i][j]).replace('*','')
        df[i][j] = str(df[i][j]).replace('-','0')

In [10]:
df['runs_scored'].unique()

array(['0', '8', '1', '52', '29', '39', '70', '24', '3', '66', '2', '9',
       '26', '11', '22', '58', '19', '12', '37', '32', '25', '18', '54',
       '31', '28', '4', '15', '60', '43', '7', '48', '57', '36', '114',
       '61', '101', '82', '13', '51', '68', '38', '6', '69', '10', '41',
       '20', '5', '44', '23', '87', '81', '86', '94', '55', '72', '90',
       '117', '95', '27', '21', '80', '77', '30', '33', '14', '108',
       '183', '106', '128', '83', '65', '47', '49', '46', '102', '115',
       '64', '42', '141', '100', '79', '209', '99', '123', '78', '62',
       '136', '56', '264', '139', '138', '107', '16', '137', '34', '63',
       '150', '171', '91', '124', '59', '85', '154', '45', '76', '96',
       '17', '104', '131', '110', '92', '71', '125', '121', '147', '113',
       '208', '88', '112', '160', '75', '129', '111', '152', '140', '157',
       '162', '133', '116', '122', '67', '103', '120', '159', '53', '119',
       '89', '35', '73', '166', '40', '74', '98', '130', 

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2021 entries, 0 to 2020
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   player_id        2021 non-null   int64 
 1   player_name      2021 non-null   object
 2   runs_scored      2021 non-null   object
 3   wickets          2021 non-null   object
 4   match_date       2021 non-null   object
 5   match_id         2021 non-null   int64 
 6   opposition_team  2021 non-null   object
 7   location         2021 non-null   object
dtypes: int64(2), object(6)
memory usage: 126.4+ KB


In [12]:
df['runs_scored'] = df['runs_scored'].astype('int')
df['wickets'] = df['wickets'].astype('int')

In [13]:
#Printing the player_id with the corresponding player_name for reference
df[['player_id','player_name']].drop_duplicates().sort_values(by='player_id',ascending=True)

,player_id,player_name
0,1,Rohit Sharma
447,2,Hardik Pandya
706,3,Shubman Gill
24,4,Virat Kohli
563,5,Shreyas Iyer
500,6,KL Rahul
40,7,Ravindra Jadeja
516,8,Shardul Thakur
445,9,Jasprit Bumrah
684,10,Mohammed Siraj


In [14]:
ind_players = "Rohit Sharma, Ishan Kishan, Virat Kohli, Shreyas Iyer, KL Rahul, Hardik Pandya, Ravindra Jadeja, Ravichandran ashwin, Kuldeep Yadav, Jasprit Bumrah, Mohammed Siraj"
ind_player_list = ind_players.split(', ')
ind_player_list

['Rohit Sharma',
 'Ishan Kishan',
 'Virat Kohli',
 'Shreyas Iyer',
 'KL Rahul',
 'Hardik Pandya',
 'Ravindra Jadeja',
 'Ravichandran ashwin',
 'Kuldeep Yadav',
 'Jasprit Bumrah',
 'Mohammed Siraj']

In [15]:
pak_players = "Babar Azam, Faqar Zaman, Imam-Ul-Haq, Mohammad Rizwan, Saud Shakeel, Iftikhar Ahmed, Mohammad Nawaz, Shadab Khan, Hasan Ali, Shaheen Afridi, Haris Rauf"
pak_players_list = pak_players.split(', ')
pak_players_list

['Babar Azam',
 'Faqar Zaman',
 'Imam-Ul-Haq',
 'Mohammad Rizwan',
 'Saud Shakeel',
 'Iftikhar Ahmed',
 'Mohammad Nawaz',
 'Shadab Khan',
 'Hasan Ali',
 'Shaheen Afridi',
 'Haris Rauf']

In [16]:
#Keeping only the playing 11 in the dataset
complete_playing_11_list = ind_player_list + pak_players_list
df1 = df.copy()
df1 = df1[df1['player_name'].isin(complete_playing_11_list)]

In [17]:
df.shape, df1.shape

((2021, 8), (1728, 8))

In [18]:
df1.head(3)

,player_id,player_name,runs_scored,wickets,match_date,match_id,opposition_team,location
0,1,Rohit Sharma,0,0,23 Jun 2007,1,Ireland,Belfast
1,1,Rohit Sharma,8,0,26 Jun 2007,2,South Africa,Belfast
2,1,Rohit Sharma,1,0,5 Oct 2007,3,Australia,Hyderabad


<h3> Approach</h3>
    <ul><li>player_name feature is redundant to player_id and therefore remove it from dataframe</li>
        <li>match_id isn't giving any additional information, except for its uniqueness to each match, instead use match date as a proximity to add a time dimension to the data. The better the performances of the players in the recent matches, the better the performance in the next match.</li>
        <li>Since this is a regression model, transform the features opposition_team and location using One Hot Encoding.</li>
        <li>Check the skewness and distibution of the target variables (runs_scored and wickets) and apply corresponding normalization techniques.</li></ul>

In [19]:
df2 = df1.copy()
df2.drop(['player_name','match_id'],axis = 1,inplace=True)

In [20]:
df2['match_date'] = pd.to_datetime(df2['match_date'])
df2['match_proximity'] = (df2['match_date'] - min(df2['match_date'])).dt.days
df2.head()

,player_id,runs_scored,wickets,match_date,opposition_team,location,match_proximity
0,1,0,0,2007-06-23,Ireland,Belfast,0
1,1,8,0,2007-06-26,South Africa,Belfast,3
2,1,1,0,2007-10-05,Australia,Hyderabad,104
3,1,52,0,2007-11-18,Pakistan,Jaipur,148
4,1,29,0,2008-02-03,Australia,Brisbane,225


In [21]:
#New Feature is created to beat the skewness of the match_date difference.
df2.sort_values(by='match_proximity',ascending=True,inplace = True)
df2.drop('match_proximity',axis=1,inplace=True)
df2['match_proximity'] = list(range(1,(df2.shape)[0] +1))
df2.rename(columns={'index':'match_proximity'},inplace = True)
df2.drop(columns = 'match_date',inplace = True)
df2.head()

,player_id,runs_scored,wickets,opposition_team,location,match_proximity
0,1,0,0,Ireland,Belfast,1
1,1,8,0,South Africa,Belfast,2
2,1,1,0,Australia,Hyderabad,3
3,1,52,0,Pakistan,Jaipur,4
4,1,29,0,Australia,Brisbane,5


In [22]:
#Performing One Hot Encoding
df3 = pd.get_dummies(data=df2,columns=['opposition_team','location'],drop_first=True)
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1728 entries, 0 to 1102
Columns: 102 entries, player_id to location_Wellington
dtypes: int64(4), uint8(98)
memory usage: 232.9 KB


In [23]:
#Splitting the predictor and target variables
#Since this is a multi output regression model, the model training is done for one target variable at a time
X = df3.drop(columns = ['runs_scored','wickets'])
y_runs = df3.runs_scored
y_wickets = df3.wickets

In [24]:
#Initially building a model for runs_scored
X_train_runs,X_test_runs,y_train_runs,y_test_runs = train_test_split(X,y_runs,test_size=0.2,random_state=42)

In [25]:
#using a library called pycaret to estimate the base models performance on the data
runs_train_data_combined = X.copy()
runs_train_data_combined['runs_scored'] = np.log1p(y_runs)
fit_data = setup(data = runs_train_data_combined,target = 'runs_scored')
best = compare_models()

,Description,Value
0,Session id,337
1,Target,runs_scored
2,Target type,Regression
3,Original data shape,"(1728, 101)"
4,Transformed data shape,"(1728, 101)"
5,Transformed train set shape,"(1209, 101)"
6,Transformed test set shape,"(519, 101)"
7,Numeric features,100
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,1.2060,2.0676,1.4354,0.2917,0.5754,0.4158,0.0290
catboost,CatBoost Regressor,1.2200,2.1763,1.4716,0.2533,0.5774,0.4418,0.1680
lightgbm,Light Gradient Boosting Machine,1.2097,2.1998,1.4811,0.2470,0.5780,0.4443,0.2300
ada,AdaBoost Regressor,1.2951,2.2413,1.4959,0.2337,0.6016,0.4108,0.0110
rf,Random Forest Regressor,1.2074,2.3319,1.5228,0.1992,0.5926,0.4616,0.0680
xgboost,Extreme Gradient Boosting,1.2679,2.5148,1.5831,0.1361,0.6031,0.4791,0.0190
br,Bayesian Ridge,1.5159,2.9404,1.7138,-0.0046,0.6951,0.4389,0.0070
en,Elastic Net,1.5164,2.9433,1.7146,-0.0054,0.6953,0.4387,0.0060
et,Extra Trees Regressor,1.3037,2.9288,1.7061,-0.0063,0.6602,0.5120,0.0800
dummy,Dummy Regressor,1.5171,2.9546,1.7178,-0.0090,0.6961,0.4385,0.0040


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [27]:
#training Gradient Boosting Regression Model
y_train_runs_norm = np.log1p(y_train_runs)
y_test_runs_norm = np.log1p(y_test_runs)
gbr = GradientBoostingRegressor()
gbr.fit(X_train_runs,y_train_runs_norm)
y_pred_runs_gbr = gbr.predict(X_test_runs)
mean_squared_error(y_test_runs_norm,y_pred_runs_gbr)

2.0951495780101808

In [30]:
#training Catboost Regressor Model
cbr = CatBoostRegressor(verbose = 0)
cbr.fit(X_train_runs,y_train_runs_norm)
y_pred_runs_cbr = cbr.predict(X_test_runs)
mean_squared_error(y_test_runs_norm,y_pred_runs_cbr)

2.2525651967397726

In [31]:
#Now building a model for wickets
X_train_wickets,X_test_wickets,y_train_wickets,y_test_wickets = train_test_split(X,y_wickets,test_size=0.2,random_state=42)

In [32]:
#using a library called pycaret to estimate the base models performance on the data
wickets_train_data_combined = X.copy()
wickets_train_data_combined['runs_scored'] = y_wickets
fit_data = setup(data = wickets_train_data_combined,target = 'runs_scored')
best = compare_models()

,Description,Value
0,Session id,7733
1,Target,runs_scored
2,Target type,Regression
3,Original data shape,"(1728, 101)"
4,Transformed data shape,"(1728, 101)"
5,Transformed train set shape,"(1209, 101)"
6,Transformed test set shape,"(519, 101)"
7,Numeric features,100
8,Preprocess,True
9,Imputation type,simple


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
gbr,Gradient Boosting Regressor,0.5200,0.7023,0.8349,0.3347,0.3895,0.4559,0.0300
lightgbm,Light Gradient Boosting Machine,0.5290,0.7309,0.8519,0.3066,0.4005,0.4950,0.2250
catboost,CatBoost Regressor,0.5360,0.7568,0.8678,0.2796,0.4022,0.5044,0.1320
rf,Random Forest Regressor,0.5062,0.8014,0.8913,0.2389,0.4043,0.5571,0.0510
xgboost,Extreme Gradient Boosting,0.5890,0.8897,0.9409,0.1457,0.4410,0.5690,0.0190
et,Extra Trees Regressor,0.5481,0.9917,0.9920,0.0592,0.4517,0.6533,0.0570
br,Bayesian Ridge,0.7948,1.0531,1.0232,0.0021,0.5154,0.6057,0.0080
en,Elastic Net,0.8002,1.0579,1.0255,-0.0023,0.5173,0.6103,0.0060
dummy,Dummy Regressor,0.8037,1.0646,1.0287,-0.0086,0.5191,0.6131,0.0060
llar,Lasso Least Angle Regression,0.8037,1.0646,1.0287,-0.0086,0.5191,0.6131,0.0060


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [33]:
#training Gradient Boosting Regression Model
gbr = GradientBoostingRegressor()
gbr.fit(X_train_wickets,y_train_wickets)
y_pred_wickets_gbr = gbr.predict(X_test_wickets)
mean_squared_error(y_test_wickets,y_pred_wickets_gbr)

0.7693824211877467

In [34]:
#training Catboost Regressor Model
cbr = CatBoostRegressor(verbose = 0)
cbr.fit(X_train_wickets,y_train_wickets)
y_pred_wickets_cbr = cbr.predict(X_test_wickets)
mean_squared_error(y_test_wickets,y_pred_wickets_cbr)

0.8015191320380446

In [54]:
columns_0 = list(submission_predictor_df.columns)
columns_0.remove('player_id')
columns_0.remove('match_proximity')

['opposition_team_Australia', 'opposition_team_Bangladesh', 'opposition_team_England', 'opposition_team_Hong Kong', 'opposition_team_India', 'opposition_team_Ireland', 'opposition_team_Nepal', 'opposition_team_Netherlands', 'opposition_team_New Zealand', 'opposition_team_Pakistan', 'opposition_team_South Africa', 'opposition_team_Sri Lanka', 'opposition_team_U.A.E.', 'opposition_team_West Indies', 'opposition_team_Zimbabwe', 'location_Adelaide', 'location_Ahmedabad', 'location_Auckland', 'location_Belfast', 'location_Bengaluru', 'location_Birmingham', 'location_Brabourne', 'location_Bridgetown', 'location_Brisbane', 'location_Bristol', 'location_Bulawayo', 'location_Canberra', 'location_Cape Town', 'location_Cardiff', 'location_Centurion', 'location_Chattogram', 'location_Chennai', 'location_Chester-le-Street', 'location_Christchurch', 'location_Colombo (RPS)', 'location_Cuttack', 'location_Dambulla', 'location_Delhi', 'location_Dharamsala', 'location_Dubai (DSC)', 'location_Dublin (Ma

In [58]:
#Creating the Submission data set with predictor variables
submission_predictor_df = pd.DataFrame(columns=X.columns)
submission_predictor_df.player_id = [i for i in X.player_id.unique()]
columns_0 = list(submission_predictor_df.columns)
columns_0.remove('player_id')
columns_0.remove('match_proximity')
for i in columns_0:
    submission_predictor_df[i] = 0
submission_predictor_df.loc[submission_predictor_df['player_id'] > 15, 'opposition_team_Pakistan'] = 1
submission_predictor_df.loc[submission_predictor_df['player_id'] < 16, 'opposition_team_India'] = 1
submission_predictor_df['location_Rajkot'] = 1
submission_predictor_df['match_proximity'] = X.match_proximity.max() + 1
submission_predictor_df

,player_id,match_proximity,opposition_team_Australia,opposition_team_Bangladesh,opposition_team_England,opposition_team_Hong Kong,opposition_team_India,opposition_team_Ireland,opposition_team_Nepal,opposition_team_Netherlands,...,location_Southampton,location_Sydney,location_Tarouba,location_Taunton,location_The Oval,location_Thiruvananthapuram,location_Vadodara,location_Visakhapatnam,location_Wankhede,location_Wellington
0,1,1729,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,1729,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,1729,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,13,1729,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,25,1729,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,16,1729,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,22,1729,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,9,1729,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,6,1729,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,24,1729,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [63]:
#Using Gradient Descent Model to predict the final output (runs_scored)
y_runs_norm = np.log1p(y_runs)
gbr_runs = GradientBoostingRegressor()
gbr_runs.fit(X,y_runs_norm)
y_pred_runs_gbr = np.round(np.exp(gbr_runs.predict(submission_predictor_df))-1)
y_pred_runs_gbr

array([38., 40., 11.,  2., 16., 32., 17.,  2., 39., 12., 24., 12., 19.,
        2., 25., 40.,  2.,  3., 12., 24.,  3.])

In [64]:
#training Gradient Boosting Regression Model
gbr_wickets = GradientBoostingRegressor()
gbr_wickets.fit(X,y_wickets)
y_pred_wickets_gbr = abs(np.round(gbr_wickets.predict(submission_predictor_df)))
y_pred_wickets_gbr

array([0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1.,
       2., 1., 0., 1.])

In [68]:
pred = []
for i, j  in zip(range(len(y_pred_runs_gbr)),range(len(y_pred_wickets_gbr))):
    pred.append([y_pred_runs_gbr[i],abs(y_pred_wickets_gbr[j])])
for i in range(len(pred)):
    for j in range(0,2):
        pred[i][j] = int(round(pred[i][j]))
final_results = pd.DataFrame(pred)
final_results.rename(columns = {0:'runs',1:'wickets'},inplace = True)

In [72]:
final_results['player_id'] = submission_predictor_df['player_id']
final_results = final_results.iloc[:,[2,0,1]]
final_results.head()

,player_id,runs,wickets
0,1,38,0
1,4,40,0
2,7,11,1
3,13,2,1
4,25,16,1


In [84]:
list(final_results.player_id.sort_values())

[1, 2, 4, 5, 6, 7, 9, 10, 11, 13, 14, 16, 17, 19, 20, 21, 22, 23, 24, 25, 28]

In [85]:
not_in_squad = [3,8,12,15,18,26,27,29,30]
not_in_squad_final = pd.DataFrame(data = not_in_squad)
not_in_squad_final.rename(columns={0:'player_id'},inplace=True)
not_in_squad_final['runs'] = 0
not_in_squad_final['wickets'] = 0

In [86]:
gbr_submission_final = pd.concat([final_results,not_in_squad_final])
gbr_submission_final.sort_values(by='player_id',inplace=True)
gbr_submission_final

,player_id,runs,wickets
0,1,38,0
10,2,24,1
0,3,0,0
1,4,40,0
15,5,40,0
8,6,39,0
2,7,11,1
1,8,0,0
7,9,2,1
16,10,2,1


In [87]:
gbr_submission_final.to_csv('submission_V1.csv',index=False)